In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
import matplotlib.pyplot as plt
import time
import os
import random
import io
from PIL import Image, ImageOps

#### Parameters

In [2]:
STILL_ALIVE_REWARD = 1
DEAD_REWARD = -1000

CROP_SHAPE = (750, 539, 1)
RESIZE_WIDTH = 180
RESIZE_HEIGHT = 137
READ_BATCH = 2

MOVES = 5
NONE = 0
RIGHT = 1
DOWN = 2
LEFT = 3
UP = 4

SHOTS_FOLDER = 'data/shots/'

GAMMA = 0.95

#### Browser functions

In [3]:
def selectLevel(l):
    xpath = '/html/body/section/div[2]/nav/p[' + str(l+1) + ']'
    
    level = browser.find_element_by_xpath(xpath)
    level.click()
    
    
def clickBoard():        
    xpath = '/html/body/section/div[2]/div'
    board = browser.find_element_by_xpath(xpath)

    try:
        board.click()
    except WebDriverException:
        print('Exception')
        return
    
    
def saveScreen(fileName):    
    state = getState()
#     if state == 'playing' or state == 'paused':    
    ss = browser.get_screenshot_as_file(SHOTS_FOLDER + fileName)
    
    
def getScreen():
    ss = browser.get_screenshot_as_png()
    im = Image.open(io.BytesIO(ss))
    
    im = preprocImg(im)
    return np.asarray(im.convert("L"))
    
    
def getState():
    xpath = '/html/body/section/div[2]'
    state = browser.find_element_by_xpath(xpath)
    c = state.get_attribute('class')
    
    return c.split(' ')[-1]


def getScore():
    state = getState()
    
    if state == 'playing' or state == 'paused':
        
        xpath = '/html/body/section/div[2]/p[1]/span'
        score = browser.find_element_by_xpath(xpath)

        if not score.text.isnumeric():
            return 0
        return int(score.text)
    
    return 0


def makeMove(m):
    xpath = '/html/body/section/div[2]/div'
    board = browser.find_element_by_xpath(xpath)

    if m == RIGHT:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_RIGHT)
    elif m == DOWN:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_DOWN)
    elif m == LEFT:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_LEFT)
    elif m == UP:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_UP)
        

def makeRandomMove():
    r = random.randint(0, 4)
    makeMove(r)
    
    return r

#### Preprocess functions

In [4]:
def getLastGameIndex():
    '''Get the index of the last current game.'''
    files = os.listdir(SHOTS_FOLDER)
    if len(files) == 0:
        return 0
    
    last = files[-1]
    
    return parseName(last)[0]


def getName(g, i, m, r):
    '''Form the file name from the board data.'''
    return str(g).zfill(3) + '_' + str(i).zfill(3) + '_' + str(m) + '_' + str(r) + '.png'


def parseName(fileName):
    '''Parse the board data from the file name.'''
    return [int(d) for d in fileName[:-4].split('_')]


def cropImgOld(fileName):
    ss = plt.imread(fileName)

    ss = ss[120:659, 273:1023, :]
    
    plt.imsave(fileName, ss)
        

def preprocImg(im):
    '''Preprocess a screenshot.'''     
    # Crop board
    im = im.crop((273, 120, 1023, 659))
    
    # Grayscale
    im = ImageOps.grayscale(im)
    
    # Binarization
    t = 127
    im = im.point(lambda x: 255 if x > t else 0)
    
    # Resize
    im = im.resize((RESIZE_WIDTH, RESIZE_HEIGHT))
    
    return im
    
    
def preprocAll(gameIndex):
    '''Preprocess all screenshots in the data folder.'''
    files = os.listdir(SHOTS_FOLDER)
    for f in files:
        g = parseName(f)[0]
        
        if g >= gameIndex:        
            fileName =  SHOTS_FOLDER + f
            im = Image.open(fileName)   
            im = preprocImg(im)
            im.save(fileName)

In [9]:
preprocAll(86)

#### Play game

In [7]:
url = 'https://playsnake.org/'

browser = webdriver.Chrome(executable_path='D:/Libraries/Drivers/chromedriver_win32/chromedriver.exe')
# browser = webdriver.Chrome(executable_path='D:/Biblioteci/Python/chromedriver_win32/chromedriver.exe')
browser.get(url) 

START_GAME = getLastGameIndex() + 1
gameIndex = START_GAME

maxGames = 100

for g in range(maxGames):
    
    # Select the level
    selectLevel(1)
    time.sleep(2)
    
    score = 0
    screenIndex = 0
    
    clickBoard()
    while getState() == 'playing' or getState() == 'paused':
        saveScreen('prev.png')
        screenIndex += 1    

        clickBoard()
        m = makeRandomMove()
        time.sleep(0.1)
        clickBoard()
        
        # Update last board  
        r = max(STILL_ALIVE_REWARD, getScore() - score) 
        fileName = getName(gameIndex, screenIndex, m, r)
        os.rename(SHOTS_FOLDER + 'prev.png', SHOTS_FOLDER + fileName)
        score = getScore()
        
    # Change reward of last board
    g, i, m, r = parseName(fileName)
    os.rename(SHOTS_FOLDER + fileName, SHOTS_FOLDER + getName(g, i, m, DEAD_REWARD))
    
    time.sleep(1)
    
    gameIndex += 1

browser.quit()

preprocAll(START_GAME)

# RIGHT = 1
# DOWN = 2
# LEFT = 3
# UP = 4

### Q model

In [5]:
from keras.models import Sequential, Model
import keras.layers
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Input
from keras.regularizers import l2
from keras.optimizers import Adam, Adadelta, RMSprop
import keras.losses as losses
from keras.models import load_model
from keras.backend import set_image_data_format
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
import glob

set_image_data_format('channels_first')

E:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Input functions

In [7]:
def readData():
    files = os.listdir(SHOTS_FOLDER)
    
    labels = np.zeros(len(files))    
    data = []
    
    for i, f in enumerate(files):
        im = Image.open(SHOTS_FOLDER + f)
        
        if f in history:
            labels[i] = history[f][2]
        else:
            labels[i] = STILL_ALIVE_REWARD
            
        data.append([np.asarray(im.convert("L"))])
        
    data = np.asarray(data)
    return data, labels


def sampleData(nSamples, propD, propR, propA):
    '''Sample the input data according to some proportions.
    
    Parameters
    ----------
    nSamples: total number of samples
    propD: proportion of samples which represent a death
    propR: proportion of samples which represent a reward
    propA: proportion of rest of samples
    '''
    
    files = os.listdir(SHOTS_FOLDER)
    
    perc = []
    nd, na, nr = (0, 0, 0)
    for f in files:
        r = parseName(f)[3]
        if r == DEAD_REWARD:
            perc.append('d')
            nd += 1
        elif r == STILL_ALIVE_REWARD:
            perc.append('a')
            na += 1
        else:
            perc.append('r')
            nr += 1
        
    n = propD * nd + propA * na + propR * nr
    pd = propD / n
    pa = propA / n
    pr = propR / n
    for i in range(len(perc)):
        if perc[i] == 'd':
            perc[i] = pd
        elif perc[i] == 'a':
            perc[i] = pa
        else:
            perc[i] = pr
        
    samples = np.random.choice(files, size=nSamples, replace=False, p=perc)
    print('Sampled data.')
    return samples


def readSamplesBasic(samples):
    '''Read samples for basic model, which just classifies if a board is finished or not.'''
    data = []
    labels = []
    
    for file in samples:
        im = Image.open(SHOTS_FOLDER + file)
        g, i, m, r = parseName(file)
        
        data.append([np.asarray(im.convert("L"))])
        labels.append(1 if r > 0 else 0)
        
    data = np.asarray(data)
    return data, labels


def getPreviousSample(file):
    '''Get the name of the previous sample.'''
    
    g, i, m, r = parseName(file)
    i -= 1
    
    if i > 0:        
        prevFile = glob.glob(SHOTS_FOLDER + str(g).zfill(3) + '_' + str(i).zfill(3) + '*')[0]
        prevFile = prevFile.replace('\\', '/')
        prevFile = prevFile.split('/')[-1]
        return prevFile      
    else:
        return -1


def getNextSample(file):
    '''Get the name of the next sample.'''
    
    g, i, m, r = parseName(file)
    i += 1
    
    prevFile = glob.glob(SHOTS_FOLDER + str(g).zfill(3) + '_' + str(i).zfill(3) + '*')
    if len(prevFile) == 1:
        prevFile = prevFile[0]
        prevFile = prevFile.replace('\\', '/')
        prevFile = prevFile.split('/')[-1]
        return prevFile
    else:
        return -1
    
    
def readSample(file):
    '''Read a single sample as an array.'''
    im = Image.open(SHOTS_FOLDER + file)
    g, i, m, r = parseName(file)

    data = np.asarray(im.convert("L"))
    label = 1 if r > 0 else 0
    
    return data, label
    
    
def readSampleMove(file):
    '''Read a single sample as an array, also returns the move.'''
    im = Image.open(SHOTS_FOLDER + file)
    g, i, m, r = parseName(file)

    data = np.asarray(im.convert("L"))
    if r == -1000:
        label = -1
    elif r > 1:
        label = r / 100
    else:
        label = 0
#     label = r
    
    return data, m, label


def readBatchSamples(samples, batch):
    '''Read samples in batches, so that we also have information about the movement.'''
    data = []
    labels = []
    
    for file in samples:        
        dataBatch = []
        
        d, label = readSample(file)
        dataBatch.append(d)
        
        # Get previous files
        prevFile = file
        for p in range(batch - 1):
            prevFile = getPreviousSample(prevFile)
            
            if prevFile == -1:
                break
                
            d, _ = readSample(prevFile)
            dataBatch.append(d)
        
        if prevFile != -1:
            dataBatch.reverse()
            data.append(dataBatch)
            labels.append(label)
        
    data = np.asarray(data)
    return data, labels   


def getStateBatches(file, batch):
    '''Get the image batches for the current and next state.'''
          
    s, _, _ = readSampleMove(file)
    
    # Get the image batch for the current state
    curBatch = []
    curBatch.append(s)
    
    # Get previous files
    prevFile = file
    for p in range(batch - 1):
        prevFile = getPreviousSample(prevFile)
        
        if prevFile == -1:
            break
            
        s, _ = readSample(prevFile)
        curBatch.append(s)
        
    if prevFile == -1:
        return -1, -1
    else:
        #Get the image batch for the next state
        nextBatch = []
        
        nextFile = getNextSample(file)
        if nextFile == -1:
            curBatch.reverse()
            return curBatch, curBatch
        
        nextState, _ = readSample(nextFile)        
        
        nextBatch.append(nextState)        
        nextBatch.extend(curBatch[:-1])
        
        curBatch.reverse()
        nextBatch.reverse()
        return curBatch, nextBatch


def readBatchSamplesMove(samples, batch):
    '''Read samples in batches, so that we also have information about the movement.
    Also reads the moves.'''
    states = []
    actions = []
    labels = []
    nextStates = []
    
    for i, file in enumerate(samples):          
        _, a, r = readSampleMove(file)
        
        curBatch, nextBatch = getStateBatches(file, batch)
        if curBatch != -1:
            states.append(curBatch)
            actions.append(a)
            labels.append(r)
            nextStates.append(nextBatch)

        if i % 100 == 0:
            print(i)
        
    states = np.asarray(states)
    actions = np.asarray(actions)
    nextStates = np.asarray(nextStates)
    labels = np.asarray(labels)
       
    # Scale labels
#     labels = np.asarray(labels).reshape(-1, 1)
#     scaler = StandardScaler()
#     scaler.fit(labels)
#     labels = scaler.transform(labels)
    
    return states, actions, labels, nextStates    
    
    
def getModelMoveInput(model, screens):
    '''Get the move from the model which takes moves as input.'''
    
    m = 0
    bestMove = 0
    bestScore = DEAD_REWARD - 1
    
    while m < MOVES:
        score = model.predict([np.asarray([screens]), np.asarray([m])])      
        
        print(m, score)
        if score > bestScore:
            bestScore = score
            bestMove = m
            
        m += 1        
    
    
def getModelMoveOutput(model, screens):
    '''Get the move from the model which produces move scores as outputs.'''
    res = model.predict(np.asarray([screens]))
    m = np.argmax(res)
    
    return m
    

In [38]:
getModelMoveOutput(model, states[2])

1

#### DNN basic

Sequential, citeste doar o imagine, nu considera actiunea.

In [11]:
def nn_model_basic():
    '''NN model for basic test.'''
    model = Sequential()
    
    model.add(Conv2D(16, (4, 4), input_shape=(1, RESIZE_HEIGHT, RESIZE_WIDTH), activation='relu', name='conv1'))
    model.add(Conv2D(16, (4, 4), activation='relu', name='conv2'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool1'))
#     model.add(Dropout(0.2))
    
    model.add(Conv2D(8, (4, 4), activation='relu', name='conv3'))
    model.add(Conv2D(8, (4, 4), activation='relu', name='conv4'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool2'))
#     model.add(Dropout(0.2))
    
    model.add(Conv2D(16, (4, 4), activation='relu', name='conv5'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool3'))
#     model.add(Dropout(0.2))
    
    model.add(Flatten(name='flat'))
    model.add(Dense(100, activation='relu', name='dense1'))
    model.add(Dropout(0.5))
    model.add(Dense(40, activation='relu', name='dense2'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid', name='output'))
    
    model.compile(loss='binary_crossentropy',
                 optimizer = 'rmsprop',
                 metrics=['accuracy'])
    
    return model


In [12]:
model = nn_model_basic()
data, labels = readSamplesBasic(sampleData(200, 10, 0.5, 0.5))
print(data.shape)
print(labels)

model.fit(data, labels, epochs=7, batch_size=10, shuffle=True)

Sampled data.
(200, 1, 137, 180)
[1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
Epoch 1/7
200/200 [==============================] - 40s 202ms/step - loss: 1.7263 - acc: 0.5800
Epoch 2/7
200/200 [==============================] - 47s 233ms/step - loss: 0.7055 - acc: 0.7150
Epoch 3/7
200/200 [==============================] - 38s 191ms/step - loss: 0.6292 - acc: 0.7800
Epoch 4/7
200/200 [==============================] - 39s 195ms/step - loss:

#### DNN seq

Sequential, citeste batch-uri de imagini, dar nu considera si actiunea.

In [25]:
def nn_model_seq():
    '''Sequential NN model.'''
    model = Sequential()
    
    model.add(Conv2D(16, (4, 4), input_shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), activation='relu', name='conv1'))
    model.add(Conv2D(16, (4, 4), activation='relu', name='conv2'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool1'))
#     model.add(Dropout(0.2))
    
    model.add(Conv2D(8, (4, 4), activation='relu', name='conv3'))
    model.add(Conv2D(8, (4, 4), activation='relu', name='conv4'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool2'))
#     model.add(Dropout(0.2))
    
    model.add(Conv2D(16, (4, 4), activation='relu', name='conv5'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool3'))
#     model.add(Dropout(0.2))
    
    model.add(Flatten(name='flat'))
    model.add(Dense(100, activation='relu', name='dense1'))
    model.add(Dropout(0.5))
    model.add(Dense(40, activation='relu', name='dense2'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid', name='output'))
    
    model.compile(loss='binary_crossentropy',
                 optimizer = 'rmsprop',
                 metrics=['accuracy'])
    
    return model


In [ ]:
model = nn_model_seq()
data, labels = readBatchSamples(sampleData(200, 10, 0.5, 0.5), READ_BATCH)
print(data.shape)
print(labels)

model.fit(data, labels, epochs=7, batch_size=10, shuffle=True)

#### DNN action input

Functional API, citeste batch-uri de imagini, considera actiunea ca un input. 

In [27]:

def nn_model_actionlast():
    '''Full NN model, using the Keras Functional API.
    Action as input, added before the final layer.'''
    screen_input = Input(shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), name='screen_input')
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv1')(screen_input)
    conv = Conv2D(16, (4, 4), activation='relu', name='conv2')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool1')(conv)
    
    conv = Conv2D(8, (4, 4), activation='relu', name='conv3')(pool)
    conv = Conv2D(8, (4, 4), activation='relu', name='conv4')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool2')(conv)
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv5')(pool)
    pool = MaxPool2D(pool_size=(2,2), name='pool3')(conv)
    
    flat = Flatten(name='flat')(pool)
    dense = Dense(100, activation='relu', name='dense1')(flat)
    drop = Dropout(0.5)(dense)
    dense = Dense(40, activation='relu', name='dense2')(drop)
    drop = Dropout(0.5)(dense)
    
    action_input = Input(shape=(1,), name='action_input')
    concat = keras.layers.concatenate([drop, action_input])
    
    output = Dense(1, activation='linear', name='output')(concat)
    
    model = Model(inputs=[screen_input, action_input], outputs=output)
    
    model.compile(loss='mean_squared_error',
                 optimizer = 'adam',
                 metrics=['accuracy'])
    
    return model



def nn_model_actionmid():
    '''Full NN model, using the Keras Functional API.
    Action as input, added after convolutional layers.'''
    screen_input = Input(shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), name='screen_input')
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv1')(screen_input)
    conv = Conv2D(16, (4, 4), activation='relu', name='conv2')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool1')(conv)
    
    conv = Conv2D(8, (4, 4), activation='relu', name='conv3')(pool)
    conv = Conv2D(8, (4, 4), activation='relu', name='conv4')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool2')(conv)
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv5')(pool)
    pool = MaxPool2D(pool_size=(2,2), name='pool3')(conv)
    
    flat = Flatten(name='flat')(pool)
    
    action_input = Input(shape=(1,), name='action_input')
    concat = keras.layers.concatenate([flat, action_input])
    
    dense = Dense(100, activation='relu', name='dense1')(concat)
    drop = Dropout(0.5)(dense)
    dense = Dense(40, activation='relu', name='dense2')(dense)
    drop = Dropout(0.5)(dense)
        
    output = Dense(1, activation='linear', name='output')(dense)
    
    model = Model(inputs=[screen_input, action_input], outputs=output)
    
    model.compile(loss='mean_squared_error',
                 optimizer = 'adam',
                 metrics=['accuracy'])
    
    return model
    

In [99]:
model = nn_model_actionlast()
states, actions, labels, nextStates = readBatchSamplesMove(sampleData(200, 5, 5, 1), READ_BATCH)
print(data.shape)

# labels = np.asarray(labels).reshape(-1, 1)
# scaler = StandardScaler()
# scaler.fit(labels)
# labels = scaler.transform(labels)
# print(labels)

model.fit([states, actions], labels, epochs=5, batch_size=32, shuffle=True)

Sampled data.
0
100
(200, 1, 137, 180)
Epoch 1/5
190/190 [==============================] - 27s 140ms/step - loss: 43.5866 - acc: 0.1737
Epoch 2/5
190/190 [==============================] - 33s 172ms/step - loss: 1.7800 - acc: 0.2789
Epoch 3/5
190/190 [==============================] - 34s 180ms/step - loss: 1.1783 - acc: 0.3263
Epoch 4/5
190/190 [==============================] - 44s 230ms/step - loss: 1.0411 - acc: 0.3842
Epoch 5/5
190/190 [==============================] - 28s 148ms/step - loss: 0.9120 - acc: 0.3579


#### DNN action output

Functional API, citeste batch-uri de imagini, considera actiunea ca output. 

In [8]:

def nn_model_actionoutput():
    '''Full NN model, using the Keras Functional API.
    Action as output, final layer has as many neurons as possible actions.'''
    screen_input = Input(shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), name='screen_input')
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv1')(screen_input)
    conv = Conv2D(16, (4, 4), activation='relu', name='conv2')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool1')(conv)
    
    conv = Conv2D(8, (4, 4), activation='relu', name='conv3')(pool)
    conv = Conv2D(8, (4, 4), activation='relu', name='conv4')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool2')(conv)
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv5')(pool)
    pool = MaxPool2D(pool_size=(2,2), name='pool3')(conv)
    
    flat = Flatten(name='flat')(pool)
    dense = Dense(100, activation='relu', name='dense1')(flat)
    drop = Dropout(rate=0.2)(dense)
    dense = Dense(40, activation='relu', name='dense2')(drop)
    drop = Dropout(rate=0.2)(dense)
    
    output = Dense(MOVES, activation='linear', name='output')(drop)
    
    model = Model(inputs=screen_input, outputs=output)
    
    model.compile(loss='mean_squared_error',
                 optimizer = 'adam',
                 metrics=['accuracy'])
    
    return model

#### Train model function

In [9]:

def trainModelOffline(states, actions, labels, nextStates, 
                      epochs, 
                      batchSize=32, 
                      file='data/models/model.h5', 
                      load=False,
                      status=True
                     ):
    '''Train a model offline. 
    Can load or generate a new model. Saves the model after each epoch.
    
    Parameters
    ----------
    states: the list of batches of images representing the current states
    actions: the list of actions taken
    labels: the list of rewards obtained
    nextStates: the list of batches of images representing the next states
    epochs: the number of training epochs
    batchSize: the training batch size
    file: the model file, used for saving and loading
    load: if true, the model is loaded from the file, otherwise a new model is generated
    status: if true, print the status of the training
    '''
    
    if load:
        model = load_model(file)  
        print('Model loaded.')
    else:
        model = nn_model_actionoutput()

    dataSize = len(states)
    
    # For each epoch
    for i in range(epochs):
        print('Epoch', i+1)
    
        batchStart = 0
        batchEnd = min(batchSize, dataSize)
    
        # For each batch
        while batchStart < batchEnd:

            targets = []
            
            b = batchStart
            # For each experience of the batch, recalculate the targets according to the Q algorithm
            while b < dataSize and b < batchEnd:
                # Get the features of the experience
                s = states[b : b+1]
                a = actions[b]
                l = labels[b]
                n = nextStates[b: b+1]
                
                target = l
                if l != DEAD_REWARD:
                    target = l + GAMMA * np.amax(model.predict(n))
                
                fullTarget = model.predict(s)
                fullTarget[0][a] = target
                
                targets.append(fullTarget[0])
                
                b += 1
            
            print(batchStart, '/', dataSize, end='\r')
                
            # Fit the model with the batch
            targets = np.asarray(targets)
            model.fit(states[batchStart:batchEnd], targets, epochs=1, verbose=0)

            batchStart = batchEnd
            batchEnd = min(batchEnd + batchSize, dataSize)
            
        print(dataSize, '/', dataSize)
        print('MSE =', evaluateLabels(model, states, actions, labels, sample=50))
        model.save(file)
        
        
    return model




In [23]:
# states, actions, labels, nextStates = readBatchSamplesMove(sampleData(1000, 5, 10, 1), READ_BATCH)
# s2, a2, l2, n2 = readBatchSamplesMove(sampleData(50, 5, 5, 1), READ_BATCH)
# print(states.shape)

model = trainModelOffline(states, actions, labels, nextStates, epochs=10)

Epoch 1
913 / 913
MSE = 0.45664884802438066
Epoch 2
913 / 913
MSE = 0.25891703720917203
Epoch 3
913 / 913
MSE = 0.3413278045064537
Epoch 4
913 / 913
MSE = 0.18662625015892254
Epoch 5
913 / 913
MSE = 0.1818491968931874
Epoch 6
913 / 913
MSE = 0.18560868383718201
Epoch 7
913 / 913
MSE = 0.25092413612235276
Epoch 8
913 / 913
MSE = 0.14643886749753737
Epoch 9
913 / 913
MSE = 0.3542502762726323
Epoch 10


KeyboardInterrupt: 


#### Evaluation

In [10]:
def evaluateLabels(model, states, actions, labels, sample=0):
    
    if sample > 0:
        sample = min(sample, len(states))
        indexes = random.sample(range(len(states)), sample)
        states = states[indexes]
        actions = actions[indexes]
        labels = labels[indexes]
        
    res = model.predict(states)   
    
    sqErr = [(res[i][actions[i]] - labels[i])**2 for i, out in enumerate(res)]
    meanSqErr = np.mean(sqErr)
    
    return meanSqErr


# def evaluateMemory(model, memory, sample=0):
    
#     if sample > 0:
#         sample = min(sample, len(memory))
#         indexes = random.sample(range(len(memory)), sample)
#         states = states[indexes]
#         actions = actions[indexes]
#         labels = labels[indexes]
        
#     res = model.predict(states)   
    
#     sqErr = [(res[i][actions[i]] - labels[i])**2 for i, out in enumerate(res)]
#     meanSqErr = np.mean(sqErr)
    
#     return meanSqErr
    
    
def evaluateTargets(model, states, actions, labels):
    return 0

In [15]:
# model.evaluate(data[30:50], labels[30:50])
evalStates = states
evalActions = actions
evalLabels = labels

# print(model.predict(evalStates))
# print(evalLabels)

evaluateLabels(model, evalStates, evalActions, evalLabels, 300)

0.23553280485184794

In [65]:
# Build pipeline

layers = []
layers.append(('nn', KerasClassifier(build_fn=nn_model_actionlast,
                                   epochs=5,
                                   batch_size=10,
                                   verbose=1)))
estimator = Pipeline(layers)


# Evaluate

data, actions, labels = readBatchSamplesMove(sampleData(200, 10, 0.5, 0.5), 2)

# print(labels)
# labels = np.asarray(labels).reshape(-1, 1)
# scaler = StandardScaler()
# scaler.fit(labels)
# labels = scaler.transform(labels)
# print(labels.reshape(labels.size))

labels = np.asarray(labels)
labels = labels.reshape((labels.size, 1))
actions = actions.reshape((actions.size, 1))
print(data.shape)
print(actions.shape)
print(labels.shape)
     
sKFold = StratifiedKFold(n_splits=2)
results = cross_val_score(estimator, [data, actions], labels, cv=sKFold, verbose=1, scoring='accuracy')

print("Results: %.5f (%.5f)" % (results.mean(), results.std()))

(193, 2, 137, 180)
(193, 1)
(193, 1)


ValueError: Found input variables with inconsistent numbers of samples: [2, 193]

### Play game with model

In [41]:
url = 'https://playsnake.org/'

browser = webdriver.Chrome(executable_path='D:/Libraries/Drivers/chromedriver_win32/chromedriver.exe')
# browser = webdriver.Chrome(executable_path='C:/Personal/Proiecte/Cod/_libs/Python/selenium_chromedriver73_win32/chromedriver.exe')
browser.get(url) 

maxGames = 1

curModel = load_model('data/models/model_s1000_e9.h5')

for g in range(maxGames):
    
    # Select the level
    selectLevel(1)
    time.sleep(2)
    
    clickBoard()
    prevScreen = getScreen()
    
    while getState() == 'playing' or getState() == 'paused':
        curScreen = getScreen()
        screens = [prevScreen, curScreen]
        
        m = getModelMoveOutput(curModel, screens)
        print(m)
        
        clickBoard()
        makeMove(m)
        clickBoard()
        
        prevScreen = curScreen
    
    time.sleep(1)

browser.quit()

1
1
4
1
4
0
0
1
1
1
1
3
4
1


### Train game online

In [114]:
url = 'https://playsnake.org/'

browser = webdriver.Chrome(executable_path='D:/Libraries/Drivers/chromedriver_win32/chromedriver.exe')
# browser = webdriver.Chrome(executable_path='C:/Personal/Proiecte/Cod/_libs/Python/selenium_chromedriver73_win32/chromedriver.exe')
browser.get(url) 

maxGames = 1

curModel = load_model('data/models/model_s1000_e9.h5')

memory = []

for g in range(maxGames):
    
    # Select the level
    selectLevel(1)
    time.sleep(2)
    
    clickBoard()
    prevScreen = getScreen()
    
    state = []
    episode = []
    score = 0
    
    while getState() == 'playing' or getState() == 'paused':
        curScreen = getScreen()
        
        # Add the new experience to the episode
        nextState = [prevScreen, curScreen]
        if state and nextState:
            exp = Experience(state, m, r, nextState)
            episode.append(exp)
        state = nextState
        
        # Make the move
        m = getModelMoveOutput(curModel, state)
        print('Move:', m, end='\r')
        
        clickBoard()
        makeMove(m)
        clickBoard()
        
        # Calculate the reward
        r = max(STILL_ALIVE_REWARD, getScore() - score) 
        
        prevScreen = curScreen
    
    time.sleep(1)
    
    # Add the episode to the memory
    rememberExp(memory, episode)
    
    # Retrain the model
    model = trainModelOnline(memory)
    

browser.quit()

Model loaded.
Sampled data.


NameError: name 'actions' is not defined

In [113]:
MAX_MEMORY = 500

class Experience:
    '''Class for an experience.'''
    def __init__(self, state, action, label, nextState):
        self.state = state
        self.action = action
        self.label = label
        self.nextState = nextState
                        
            
def rememberExp(memory, exps):
    '''Add new experiences to the memory.'''
    for e in exps:
        if len(memory) == MAX_MEMORY:
            memory.pop(0)
        memory.append(e)


def sampleMemory(memory, nSamples, propD, propR, propA):
    '''Sample experiences from the memroy according to some proportions.
    
    Parameters
    ----------
    memory: the remembered experiences
    nSamples: total number of samples
    propD: proportion of samples which represent a death
    propR: proportion of samples which represent a reward
    propA: proportion of rest of samples
    '''
    
    nSamples = min(len(memory), nSamples)
        
    perc = []
    nd, na, nr = (0, 0, 0)
    for exp in memory:
        r = exp.label
        if r == DEAD_REWARD:
            perc.append('d')
            nd += 1
        elif r == STILL_ALIVE_REWARD:
            perc.append('a')
            na += 1
        else:
            perc.append('r')
            nr += 1
        
    n = propD * nd + propA * na + propR * nr
    pd = propD / n
    pa = propA / n
    pr = propR / n
    for i in range(len(perc)):
        if perc[i] == 'd':
            perc[i] = pd
        elif perc[i] == 'a':
            perc[i] = pa
        else:
            perc[i] = pr
        
    samples = np.random.choice(memory, size=nSamples, replace=False, p=perc)
    print('Sampled data.')
    return samples


def trainModelOnline(memory, 
                      batchSize=32, 
                      file='data/models/model.h5', 
                      load=True,
                      status=True
                     ):
    '''Train a model online, from the experience memory. 
    Can load or generate a new model. Saves the model at the end.
    
    Parameters
    ----------
    memory: the experience memory
    batchSize: the training batch size
    file: the model file, used for saving and loading
    load: if true, the model is loaded from the file, otherwise a new model is generated
    status: if true, print the status of the training
    '''
    
    if load:
        model = load_model(file)  
        print('Model loaded.')
    else:
        model = nn_model_actionoutput()
        
    # Sample experiences from the memory
    batch = sampleMemory(memory, batchSize, 5, 10, 1)
    batchSize = len(batch)

    states = []
    targets = []
    # For each experience
    for e in batch:        
        target = e.label
        if e.label != DEAD_REWARD:
            target = e.label + GAMMA * np.amax(model.predict(np.asarray([e.nextState])))
        
        fullTarget = model.predict(np.asarray([e.state]))
        fullTarget[0][e.action] = target
        
        targets.append(fullTarget[0])
        states.append(e.state)
               
    # Fit the model with the batch
    targets = np.asarray(targets)
    model.fit(np.asarray(states), targets, epochs=1, verbose=0)
        
#     print('MSE =', evaluateLabels(model, states, actions, labels, sample=50))
    model.save(file)
                
    return model


In [102]:
# exps = [3, 1]
# rememberExp(memory, exps)
# print(memory)

# # sampleMemory(memory, 5, 5, 10, 1)

# print(memory[0].state)
print(np.asarray(memory[0].nextState))
l = [1, 2]
print([l])


# Retrain the model
# model = trainModelOnline(memory)

[[[  0   0   0 ...   0   0   0]
  [  0 255 255 ... 255 255   0]
  [  0 255 255 ... 255 255   0]
  ...
  [  0 255 255 ... 255 255   0]
  [  0 255 255 ... 255 255   0]
  [  0   0   0 ...   0   0   0]]

 [[  0   0   0 ...   0   0   0]
  [  0 255 255 ... 255 255   0]
  [  0 255 255 ... 255 255   0]
  ...
  [  0 255 255 ... 255 255   0]
  [  0 255 255 ... 255 255   0]
  [  0   0   0 ...   0   0   0]]]
[[1, 2]]
